In [1]:
import numpy as np
import pandas as pd
import sqlite3 as sql

#from pandasql import sqldf

In [2]:
#pysqldf = lambda q: sqldf(q, globals())
#pysqldf

In [3]:
adult_names  = pd.read_table('adult_names.txt').iloc[91:]
adult_names.columns = ['name']
column_name = adult_names['name'].str.split(':', expand= True)[0]
columns =  column_name.iloc[1:] 
columns = columns.append(column_name.iloc[0:1], ignore_index=True)
columns

0                age
1          workclass
2             fnlwgt
3          education
4      education-num
5     marital-status
6         occupation
7       relationship
8               race
9                sex
10      capital-gain
11      capital-loss
12    hours-per-week
13    native-country
14      >50K, <=50K.
Name: 0, dtype: object

In [4]:
#conn = sql.connect('sqladb')
#query = "create table adult ('" + "', '".join(list(c for c in column_name)) + "' );"
#conn.execute(query)

#conn.execute('drop table adult')

In [5]:
adult = pd.read_csv('adult_data.csv', header= None)
adult.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [32]:
adult.columns = [col for col in columns]
adult.head()
#adult.describe()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,">50K, <=50K."
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [8]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///sqldb.db')
engine.execute('DROP table adult')
adult.to_sql('adult', con=engine)

### 1. Select 10 records from the adult sqladb

In [9]:
rows = engine.execute("SELECT * FROM adult Limit 10").fetchall()
for row in rows:
    print(row, '\n')

(0, 39, ' State-gov', 77516, ' Bachelors', 13, ' Never-married', ' Adm-clerical', ' Not-in-family', ' White', ' Male', 2174, 0, 40, ' United-States', ' <=50K') 

(1, 50, ' Self-emp-not-inc', 83311, ' Bachelors', 13, ' Married-civ-spouse', ' Exec-managerial', ' Husband', ' White', ' Male', 0, 0, 13, ' United-States', ' <=50K') 

(2, 38, ' Private', 215646, ' HS-grad', 9, ' Divorced', ' Handlers-cleaners', ' Not-in-family', ' White', ' Male', 0, 0, 40, ' United-States', ' <=50K') 

(3, 53, ' Private', 234721, ' 11th', 7, ' Married-civ-spouse', ' Handlers-cleaners', ' Husband', ' Black', ' Male', 0, 0, 40, ' United-States', ' <=50K') 

(4, 28, ' Private', 338409, ' Bachelors', 13, ' Married-civ-spouse', ' Prof-specialty', ' Wife', ' Black', ' Female', 0, 0, 40, ' Cuba', ' <=50K') 

(5, 37, ' Private', 284582, ' Masters', 14, ' Married-civ-spouse', ' Exec-managerial', ' Wife', ' White', ' Female', 0, 0, 40, ' United-States', ' <=50K') 

(6, 49, ' Private', 160187, ' 9th', 5, ' Married-spou

### 2. Show me the average hours per week of all men who are working in private sector

In [34]:
rows = engine.execute('''SELECT avg("hours-per-week") from adult WHERE workclass=' Private' ''').fetchall()         # Private has extra leading space
#rows = engine.execute("SELECT sql FROM sqlite_master WHERE name='adult'")

#print(rows)        #list

for row in rows:
    print(row, '\n')

(40.267095523440254,) 



### 3. Show me the frequency table for education, occupation and relationship, separately

In [37]:
edu_rows = engine.execute('''Select education, count(*) from adult group by education''')
for row in edu_rows:
    print(row, '\n')

(' 10th', 933) 

(' 11th', 1175) 

(' 12th', 433) 

(' 1st-4th', 168) 

(' 5th-6th', 333) 

(' 7th-8th', 646) 

(' 9th', 514) 

(' Assoc-acdm', 1067) 

(' Assoc-voc', 1382) 

(' Bachelors', 5355) 

(' Doctorate', 413) 

(' HS-grad', 10501) 

(' Masters', 1723) 

(' Preschool', 51) 

(' Prof-school', 576) 

(' Some-college', 7291) 



In [38]:
ocu_rows = engine.execute('''Select occupation, count(*) from adult group by occupation''')
for row in ocu_rows:
    print(row, '\n')

(' ?', 1843) 

(' Adm-clerical', 3770) 

(' Armed-Forces', 9) 

(' Craft-repair', 4099) 

(' Exec-managerial', 4066) 

(' Farming-fishing', 994) 

(' Handlers-cleaners', 1370) 

(' Machine-op-inspct', 2002) 

(' Other-service', 3295) 

(' Priv-house-serv', 149) 

(' Prof-specialty', 4140) 

(' Protective-serv', 649) 

(' Sales', 3650) 

(' Tech-support', 928) 

(' Transport-moving', 1597) 



In [39]:
rel_rows = engine.execute('''Select relationship, count(*) from adult group by relationship''')
for row in rel_rows:
    print(row, '\n')

(' Husband', 13193) 

(' Not-in-family', 8305) 

(' Other-relative', 981) 

(' Own-child', 5068) 

(' Unmarried', 3446) 

(' Wife', 1568) 



### 4. Are there any people who are married, working in private sector and having a masters degree

In [49]:
q = '''Select * from adult 
        where "marital-status" in (' Married-civ-spouse', ' Married-spouse-absent', ' Separated', ' Married-AF-spouse') 
        and workclass = ' Private'
        and education = ' Masters' '''

rows = engine.execute(q)
for row in rows:
    print(row, '\n')

(5, 37, ' Private', 284582, ' Masters', 14, ' Married-civ-spouse', ' Exec-managerial', ' Wife', ' White', ' Female', 0, 0, 40, ' United-States', ' <=50K') 

(87, 33, ' Private', 202051, ' Masters', 14, ' Married-civ-spouse', ' Prof-specialty', ' Husband', ' White', ' Male', 0, 0, 50, ' United-States', ' <=50K') 

(100, 76, ' Private', 124191, ' Masters', 14, ' Married-civ-spouse', ' Exec-managerial', ' Husband', ' White', ' Male', 0, 0, 40, ' United-States', ' >50K') 

(188, 31, ' Private', 99928, ' Masters', 14, ' Married-civ-spouse', ' Prof-specialty', ' Wife', ' White', ' Female', 0, 0, 50, ' United-States', ' <=50K') 

(198, 35, ' Private', 138992, ' Masters', 14, ' Married-civ-spouse', ' Prof-specialty', ' Other-relative', ' White', ' Male', 7298, 0, 40, ' United-States', ' >50K') 

(311, 34, ' Private', 142897, ' Masters', 14, ' Married-civ-spouse', ' Exec-managerial', ' Husband', ' Asian-Pac-Islander', ' Male', 7298, 0, 35, ' Taiwan', ' >50K') 

(361, 62, ' Private', 270092, ' M

### 5. What is the average, minimum and maximum age group for people working in different sectors

In [50]:
q = ''' select workclass, avg(age), min(age), max(age) from adult group by workclass'''

rows = engine.execute(q)
for row in rows:
    print(row, '\n')

(' ?', 40.960239651416124, 17, 90) 

(' Federal-gov', 42.590625, 17, 90) 

(' Local-gov', 41.75107501194458, 17, 90) 

(' Never-worked', 20.571428571428573, 17, 30) 

(' Private', 36.797585477617204, 17, 90) 

(' Self-emp-inc', 46.01702508960574, 17, 84) 

(' Self-emp-not-inc', 44.96969696969697, 17, 90) 

(' State-gov', 39.43605546995377, 17, 81) 

(' Without-pay', 47.785714285714285, 19, 72) 



### 6. Calculate age distribution by country

In [62]:
q = ''' select "native-country", age, count(*) from adult group by "native-country", age'''

rows = engine.execute(q)
for row in rows:
    print(row, '\n')

(' ?', 18, 6) 

(' ?', 19, 5) 

(' ?', 20, 8) 

(' ?', 21, 10) 

(' ?', 22, 10) 

(' ?', 23, 6) 

(' ?', 24, 14) 

(' ?', 25, 11) 

(' ?', 26, 18) 

(' ?', 27, 15) 

(' ?', 28, 19) 

(' ?', 29, 12) 

(' ?', 30, 19) 

(' ?', 31, 18) 

(' ?', 32, 16) 

(' ?', 33, 13) 

(' ?', 34, 23) 

(' ?', 35, 15) 

(' ?', 36, 23) 

(' ?', 37, 22) 

(' ?', 38, 20) 

(' ?', 39, 19) 

(' ?', 40, 11) 

(' ?', 41, 21) 

(' ?', 42, 23) 

(' ?', 43, 14) 

(' ?', 44, 8) 

(' ?', 45, 17) 

(' ?', 46, 14) 

(' ?', 47, 12) 

(' ?', 48, 5) 

(' ?', 49, 4) 

(' ?', 50, 12) 

(' ?', 51, 7) 

(' ?', 52, 9) 

(' ?', 53, 4) 

(' ?', 54, 5) 

(' ?', 55, 12) 

(' ?', 56, 10) 

(' ?', 57, 5) 

(' ?', 58, 2) 

(' ?', 59, 3) 

(' ?', 60, 5) 

(' ?', 61, 3) 

(' ?', 62, 3) 

(' ?', 63, 5) 

(' ?', 64, 5) 

(' ?', 65, 3) 

(' ?', 66, 2) 

(' ?', 68, 2) 

(' ?', 69, 1) 

(' ?', 70, 1) 

(' ?', 72, 2) 

(' ?', 76, 1) 

(' ?', 81, 1) 

(' ?', 90, 1) 

(' Cambodia', 18, 1) 

(' Cambodia', 25, 1) 

(' Cambodia', 27, 2) 

(' Camb

(' Japan', 31, 1) 

(' Japan', 32, 1) 

(' Japan', 33, 4) 

(' Japan', 34, 2) 

(' Japan', 35, 2) 

(' Japan', 36, 3) 

(' Japan', 37, 3) 

(' Japan', 38, 2) 

(' Japan', 39, 2) 

(' Japan', 40, 2) 

(' Japan', 41, 5) 

(' Japan', 43, 1) 

(' Japan', 44, 2) 

(' Japan', 45, 3) 

(' Japan', 46, 1) 

(' Japan', 47, 4) 

(' Japan', 48, 2) 

(' Japan', 50, 1) 

(' Japan', 52, 1) 

(' Japan', 53, 2) 

(' Japan', 54, 1) 

(' Japan', 56, 1) 

(' Japan', 61, 1) 

(' Laos', 19, 2) 

(' Laos', 23, 1) 

(' Laos', 24, 1) 

(' Laos', 27, 2) 

(' Laos', 29, 2) 

(' Laos', 30, 1) 

(' Laos', 32, 1) 

(' Laos', 36, 2) 

(' Laos', 42, 1) 

(' Laos', 44, 1) 

(' Laos', 49, 1) 

(' Laos', 50, 1) 

(' Laos', 53, 1) 

(' Laos', 56, 1) 

(' Mexico', 17, 6) 

(' Mexico', 18, 5) 

(' Mexico', 19, 13) 

(' Mexico', 20, 17) 

(' Mexico', 21, 24) 

(' Mexico', 22, 24) 

(' Mexico', 23, 31) 

(' Mexico', 24, 25) 

(' Mexico', 25, 38) 

(' Mexico', 26, 20) 

(' Mexico', 27, 26) 

(' Mexico', 28, 33) 

(' Mexico', 


('USA', 69, 76) 

('USA', 70, 62) 

('USA', 71, 46) 

('USA', 72, 40) 

('USA', 73, 36) 

('USA', 74, 28) 

('USA', 75, 27) 

('USA', 76, 26) 

('USA', 77, 15) 

('USA', 78, 11) 

('USA', 79, 15) 

('USA', 80, 16) 

('USA', 81, 9) 

('USA', 82, 5) 

('USA', 83, 5) 

('USA', 84, 5) 

('USA', 85, 2) 

('USA', 86, 1) 

('USA', 88, 3) 

('USA', 90, 31) 



### 7. Compute a new column as 'Net-Capital-Gain' from the two columns 'capital-gain' and 'capital-loss'

In [55]:
q = '''select ("capital-gain" - "capital-loss") as "Net-Capital-Gain" from adult'''

rows = engine.execute(q)
for row in rows:
    print(row, '\n')

(2174,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(14084,) 

(5178,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(-2042,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(-1408,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(-1902,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(5013,) 

(2407,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(14344,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(-1573,) 

(0,) 

(0,) 

(-1902,) 

(0,) 

(0,) 

(0,) 

(0,) 

(15024,) 

(0,) 

(0,) 

(0,) 

(7688,) 

(34095,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(-1887,) 

(4064,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(-1719,) 

(0,) 

(0,) 

(0,) 

(0,) 

(-1762,) 

(4386,) 



(0,) 

(1471,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(3674,) 

(0,) 

(0,) 

(2329,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(3411,) 

(0,) 

(7688,) 

(0,) 

(0,) 

(0,) 

(-1848,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(99999,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(20051,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(-1564,) 

(0,) 

(0,) 

(0,) 

(-1887,) 

(-1762,) 

(0,) 

(0,) 

(0,) 

(5013,) 

(0,) 

(2105,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(4386,) 

(3411,) 

(0,) 

(-1887,) 

(0,) 

(0,) 

(0,) 

(594,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(-1672,) 

(0,) 

(0,) 

(0,) 

(-1902,) 

(0,) 

(5013,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(-1977,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(14084,) 

(-1726,) 

(0,) 

(-1485,) 

(7688,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(-2444,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(7688,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(7298,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(-2392,) 

(0,) 

(0,) 

(-1672,) 

(0,) 

(2062,) 

(-1138,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(-1902,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(-1740,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(-1669,) 

(0,) 

(0,) 


(0,) 

(0,) 

(0,) 

(0,) 

(27828,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(15024,) 

(0,) 

(0,) 

(0,) 

(0,) 

(7688,) 

(7298,) 

(0,) 

(0,) 

(0,) 

(0,) 

(15024,) 

(0,) 

(0,) 

(0,) 

(0,) 

(15024,) 

(0,) 

(0,) 

(0,) 

(0,) 

(8614,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(-1902,) 

(0,) 

(-1539,) 

(0,) 

(0,) 

(0,) 

(-1848,) 

(0,) 

(0,) 

(0,) 

(5013,) 

(0,) 

(0,) 

(0,) 

(0,) 

(99999,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(-1762,) 

(0,) 

(0,) 

(5178,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(15024,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(99999,) 

(0,) 

(0,) 

(-880,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 




(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(-1887,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(-1876,) 

(0,) 

(0,) 

(0,) 

(0,) 

(-1977,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(10520,) 

(7688,) 

(0,) 

(0,) 

(0,) 

(-2258,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(-2051,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(4064,) 

(0,) 

(20051,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(3418,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,

...

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(-1380,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(15024,) 

(0,) 

(99999,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(-1602,) 

(0,) 

(0,) 

(0,) 

(5178,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(4064,) 

(0,) 

(0,) 

(0,) 

(-1887,) 

(0,) 

(0,) 

(0,) 

(0,) 

(13550,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(7298,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(-1485,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(-1258,) 

(0,) 

(0,) 

(0,) 

(0,) 

(-2001,) 

(0,) 

(0,) 

(0,) 

(0,) 

(14084,) 

(0,) 

(0,) 

(5013,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(-2

(0,) 

(0,) 

(7298,) 

(7298,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(-1902,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(2977,) 

(0,) 

(0,) 

(15024,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(-1902,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(3781,) 

(0,) 

(0,) 

(0,) 

(7298,) 

(0,) 

(-1741,) 

(0,) 

(0,) 

(0,) 

(0,) 

(-2377,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(7896,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(-1887,) 

(0,) 

(0,) 

(13550,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(25236,) 

(-2258,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(5013,) 

(0,) 

(-2377,) 

(

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(7298,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(-1719,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(10520,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(7688,) 

(0,) 

(0,) 

(27828,) 

(0,) 

(7298,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(2174,) 

(0,) 

(3432,) 

(0,) 

(0,) 

(0,) 

(-1902,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(-1887,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(6497,) 

(0,) 

(0,) 

(0,) 

(0,) 

(-2415,) 

(15024,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(7298,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(2463,) 

(0,) 

(99999,) 

(7688,) 

(0,) 

(7298,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 



(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(-1594,) 

(2829,) 

(0,) 

(2228,) 

(0,) 

(0,) 

(3137,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(6497,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(8614,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(4416,) 

(0,) 

(0,) 

(0,) 

(3887,) 

(3464,) 

(99999,) 

(0,) 

(0,) 

(0,) 

(0,) 

(-1902,) 

(3456,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(4386,) 

(0,) 

(0,) 

(15024,) 

(0,) 

(0,) 

(9386,) 

(0,) 

(0,) 

(3103,) 

(0,) 

(0,) 

(0,) 

(-1848,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(-1977,) 

(0,) 

(0,) 

(0,) 

(15024,) 

(0,) 

(0,) 

(0,) 

(3103,) 

(0,) 

(0,) 

(-2559,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(15831,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(6849,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(15024,) 

(0,) 

(2653,) 

(0,) 

(0,) 

(27828,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(1471,) 

(0,) 

(0,) 

(0,) 

(0,) 

(-1741,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(7298,) 

(-1887,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(-1902,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(7443,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(0,) 

(-1887,) 

(0,) 

(-160